# Oefening deel 4: Energiestromen Visualiseren met Sankey Diagram

> **Workshop Atic 4D**  
> **Lesgevers:** Lien De Backer & Jakob De Vreese

In deze oefening visualiseren we de energiestromen van het gebouw Dunant 1 aan de hand van een **Sankey diagram**. Dit type diagram is perfect om energiebalansen overzichtelijk weer te geven.

## 🎯 Leerdoelen

Na deze oefening kun je:
- Sankey diagrammen maken met Matplotlib en Plotly
- Energiestromen logisch structureren (input → conversie → output → verbruik)
- Energiebalansen opstellen en valideren
- Verliezen en rendementen visualiseren
- Interactieve visualisaties maken voor rapportage

## 📊 Datasets

We gebruiken volgende datasets (data van 2020-2025):
1. **Gasteller** - gasverbruik voor gasketel (kWh)
2. **Calorieteller gasketel** - warmte-opwekking gasketel (kWh)
3. **Calorieteller injectie gasketel** - Injectie vna ketel in lage-temperatuur circuit (kWh)
4. **Elektriciteit warmtepomp** - elektrisch verbruik WP (kWh)
5. **Calorieteller warmtepomp** - warmte/koude opwekking WP (kWh)
6. **BEO-veld** - energie naar/van bodem (kWh)
7. **Calorieteller afgifte** - warmte naar verschillende systemen:
   - Vloerverwarming
   - Luchtgroepen
   - Radiatoren (BEO)

## 🏗️ Structuur

1. Packages importeren
2. Data inladen en opschonen
3. Basis Sankey diagram (eenvoudig voorbeeld)
4. Energiebalans berekenen voor het gebouw
5. Volledig Sankey diagram met alle energiestromen
6. Interactief Sankey diagram met Plotly
7. Tijdsafhankelijke analyse (maandelijks/jaarlijks)

## 1️⃣ Packages Importeren

In [ ]:
# VUL AAN: importeer de standaard libraries

## 2️⃣ Hulpfuncties voor Data Inladen

We hergebruiken de hulpfunctie uit de vorige oefeningen om GBS-export bestanden in te laden.

In [ ]:
def load_gbs_export(filepath, energy_cols_keywords, date_format='%m/%d/%y'):
    """
    Laad een tab-gescheiden GBS export bestand in.
    
    Parameters:
    -----------
    filepath : str
        Pad naar het bestand
    energy_cols_keywords : dict
        Dict met {nieuwe_naam: zoekterm} voor kolommen
    date_format : str
        Format van de datum in het bestand
        
    Returns:
    --------
    pd.DataFrame met DatetimeIndex en hernoemde kolommen
    """
    # Inladen
    df = pd.read_csv(filepath, sep='\t')
    
    # Verwijder Unnamed kolommen
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # Zoek en hernoem energie kolommen
    rename_dict = {}
    for new_name, keyword in energy_cols_keywords.items():
        matching_cols = [c for c in df.columns if keyword.lower() in c.lower()]
        if matching_cols:
            rename_dict[matching_cols[0]] = new_name
    
    df = df.rename(columns=rename_dict)
    
    # Vind Time kolom
    time_col = 'Time' if 'Time' in df.columns else [c for c in df.columns if 'time' in c.lower()][0]
    
    # Parse datum (alleen deel voor komma)
    df['date'] = (
        df[time_col]
        .astype(str)
        .str.split(',', n=1)
        .str[0]
        .str.strip()
    )
    
    df['date'] = pd.to_datetime(df['date'], format=date_format, errors='coerce')
    df = df.dropna(subset=['date'])
    
    # Zet index en drop originele time kolom
    df = df.set_index('date').drop(columns=[time_col])
    
    # Selecteer alleen de hernoemde kolommen
    valid_cols = [col for col in energy_cols_keywords.keys() if col in df.columns]
    
    return df[valid_cols]

print("✓ Hulpfunctie gedefinieerd")

## 3️⃣ Data Inladen

We laden alle benodigde datasets in. Voor deze oefening focussen we op **jaarlijkse totalen** om de energiestromen overzichtelijk te houden.

In [ ]:
# 3.1 Gasteller (primaire energie-input)
gasteller = pd.read_csv('data/gasteller.txt', sep='\t', header=0)
gasteller = gasteller.iloc[:,:-1] # Laatse kolom laten vallen
gasteller.columns = ['datum', 'gastellerstand'] # Kolommen hernoemen
gasteller['datum'] = pd.to_datetime(
    gasteller['datum'].str.split(',').str[0],
    format='%m/%d/%y',
    errors='coerce'
) # Omzetten naar datetime
gasteller = gasteller.set_index('datum').sort_index() # Datum als index instellen
gasteller = gasteller / 100  # Correctie voor komma-fout

print(gasteller.head())

gasverbruik = gasteller.diff().rename(columns={'gastellerstand': 'gasverbruik'})
gasverbruik = gasverbruik.clip(lower=0)  # Negatieve waarden op 0
gasverbruik = gasverbruik[gasverbruik < 1000]  # Verwijder uitschieters
gasverbruik_kwh = gasverbruik * 11.4  # m³ → kWh

print(f"✓ Gasteller: {len(gasverbruik_kwh)} dagen")

In [ ]:
# VUL AAN: Plot gasverbruik_kwh


In [ ]:
# 3.2 Calorieteller gasketel (warmte-output gasketel)
# TODO -> laad cal_gasketel (data/calorieteller_gasketel.txt) 

# VUL AAN: maak een variabele cal_gasketel_dag met de dagelijkse verbruiken en zet om van Wh naar kWh

# VUL AAN: haal negatieve waarden uit de dataset cal_gasketel_dag (dit kan met .clip(lower=0))


print(f"✓ Calorieteller gasketel: ### VUL AAN ### dagen")
print(cal_gasketel.head())

In [ ]:
# VUL AAN: plot de dagelijkse opwekking


In [ ]:
# 3.3 Elektriciteit warmtepomp (primaire energie-input)
elek = pd.read_csv('data/elek_WP_1102.csv', parse_dates=['datum'], index_col='datum')
elek.columns = elek.columns.str.replace('\ufeff', '', regex=False).str.strip()
elek = elek.sort_index()

print(f"✓ Elektriciteit WP: {len(elek)} metingen")
print(elek.head())

In [ ]:
# VUL AAN: plot de elektrische meterstanden


In [ ]:
# 3.4 Calorieteller warmtepomp (warmte/koude output WP)
# TODO -> laad cal_wp in met data/calorieteller_wp.txt en 'wp_warm': 'energieverbruik warm' via load_gbs_export

# VUL AAN: maak een cal_wp_dag met de dagelijkse verbruiken waarvan de negatieve waarden er uit worden gehaald


print(f"✓ Calorieteller WP: ### VUL AAN ### dagen")
print(cal_wp.head())

In [ ]:
# VUL AAN: plot cal_wp_dag


In [ ]:
# 3.5 BEO-veld (energie naar/van bodem)
# TODO -> maak cal_beo waarbij via load_gbs_export de dataset data/calorieteller_beo.txt wordt ingelezen met 'beo_afname': 'koud', 'beo_injectie': 'warm'


# VUL AAN: Converteer van MWh naar kWh indien nodig


# VUL AAN: maak de dataset cal_beo_dag (vergeet niet te 'clippen')


print(f"✓ BEO-veld: ### VUL AAN ### dagen")
print(cal_beo.head())

In [ ]:
# VUL AAN: Plot de beo afname en injectie


In [ ]:
# VUL AAN: laad ook de injectie van de gasketel in de lage temperaturen in en destileer ook de dagelijkse waarden hieruit.


In [ ]:
# VUL AAN: plot de dagelijkse waarden van de injectie

In [ ]:
# 3.6 Afgiftesystemen (waar gaat de warmte naartoe?)
# cal_vloer = 

# cal_vloer_dag = 

# cal_lucht = 

# cal_lucht_dag = 

# cal_radiatoren_lg_ht = ketel - injectie

print(f"✓ Afgiftesystemen ingeladen")

In [ ]:
# PLOT DE INGELADE AFGIFTESYSTEMEN

## 4️⃣ Jaarlijkse Energiebalans Berekenen

Voor een overzichtelijk Sankey diagram werken we met **jaarlijkse totalen**. We berekenen de totale energie-in, conversie en -uit voor het jaar 2024.

In [ ]:
# Selecteer jaar 2023 voor analyse
year = 2023
mask = lambda df: df.index.year == year

# Bereken jaarlijkse totalen (in MWh voor leesbaarheid)
energie_2023 = {
    # INPUT (primaire energie)
    'Gas': gasverbruik_kwh[mask(gasverbruik_kwh)]['gasverbruik'].sum() / 1000,
    
    # CONVERSIE (warmte-opwekking)
    'Gasketel_warmte': cal_gasketel_dag[mask(cal_gasketel_dag)]['gasketel_warm'].sum() / 1000,
    'Injectie_lt': cal_inj_dag[mask(cal_inj_dag)]['injectie'].sum() / 1000,
    'WP_warmte': cal_wp_dag[mask(cal_wp_dag)]['wp_warm'].sum() / 1000,
    
    # BEO (energieopslag)
    'BEO_uit': cal_beo_dag[mask(cal_beo_dag)]['beo_afname'].sum() / 1000,
    'BEO_in': cal_beo_dag[mask(cal_beo_dag)]['beo_injectie'].sum() / 1000,
    
    # AFGIFTE (waar gaat warmte naartoe)
    'Vloerverwarming_warm': cal_vloer_dag[mask(cal_vloer_dag)]['vloerverw_warm'].sum() / 1000,
    'Vloerverwarming_koud': cal_vloer_dag[mask(cal_vloer_dag)]['vloerverw_koud'].sum() / 1000,
    'Luchtgroepen_warm': cal_lucht_dag[mask(cal_lucht_dag)]['luchtgroep_warm'].sum(),
    'Luchtgroepen_koud': cal_lucht_dag[mask(cal_lucht_dag)]['luchtgroep_koud'].sum(),
    'Radiatoren_Lucht': (cal_gasketel_dag[mask(cal_gasketel_dag)]['gasketel_warm'].sum() - cal_inj_dag[mask(cal_inj_dag)]['injectie'].sum()) / 1000
}

# Print overzicht
print(f"\n📊 ENERGIEBALANS ### VUL AAN ### (MWh)")
print("="*50)
print("\n🔌 PRIMAIRE ENERGIE (INPUT)")
print(f"  Gas verbruik:              ### VUL AAN ### MWh")

print("\n⚙️  CONVERSIE (WARMTE-OPWEKKING)")
print(f"  Gasketel warmte:           ### VUL AAN ### MWh")
print(f"  WP warmte:                 ### VUL AAN ### MWh")
print(f"  Injectie LT:               ### VUL AAN ### MWh")

print("\n🏔️  BEO-VELD")
print(f"  Naar bodem (laden):        ### VUL AAN ### MWh")
print(f"  Uit bodem (ontladen):      ### VUL AAN ### MWh")
print(f"  Netto balans:              ### VUL AAN ### MWh")

print("\n🏠 AFGIFTE (VERBRUIK)")
print(f"  Vloerverwarming (verwarmen):        ### VUL AAN ### MWh")
print(f"  Vloerverwarming (koelen):           ### VUL AAN ### MWh")
print(f"  Luchtgroepen (verwarmen):           ### VUL AAN ### MWh")
print(f"  Luchtgroepen (koelen):              ### VUL AAN ### MWh")
print(f"  Totaal radiatoren en nvwb lucht:    ### VUL AAN ### MWh")
print('-'*50)
print(f"  Totaal afgifte (verwarmen):         ### VUL AAN ### MWh")
print(f"  Totaal afgifte (koelen):            ### VUL AAN ### MWh")

## 5️⃣ Basis Sankey Diagram (Eenvoudig Voorbeeld)

Voordat we het volledige energiesysteem visualiseren, maken we eerst een **eenvoudig voorbeeld** om de werking van Sankey diagrammen te begrijpen.

### 🎓 Hoe werkt een Sankey diagram?

Een Sankey diagram visualiseert **stromen** tussen **nodes** (knooppunten). De **breedte** van de pijlen is proportioneel met de **hoeveelheid** die stroomt.

**Basis concepten:**
- **Flows**: Een lijst met waarden (positief = naar rechts, negatief = naar links)
- **Labels**: Namen voor elke stroom
- **Orientations**: Richting van de stroom (0 = rechts, 1 = naar boven, -1 = naar beneden)

**Voorbeeld:** Een simpel systeem met gas → ketel → warmte

In [ ]:
# Eenvoudig voorbeeld: Gas → Gasketel → Warmte
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])

sankey = Sankey(ax=ax, scale=0.01, offset=0.2, head_angle=150)

# Flows: positief = input, negatief = output
# Gas in → Warmte uit + Verlies
sankey.add(
    flows=[100, -85, -15],  # 100 MWh gas → 85 MWh warmte + 15 MWh verlies
    labels=['Gas\n100 MWh', 'Warmte\n85 MWh', 'Verlies\n15 MWh'],
    orientations=[0, 0, 1],  # Rechts, rechts, naar boven
    facecolor='lightblue',
    edgecolor='black',
    alpha=0.7
)

diagrams = sankey.finish()
plt.title('Eenvoudig Sankey Diagram: Gasketel\n(Rendement 85%)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Interpretatie:")
print("   - Breedte pijl = hoeveelheid energie")
print("   - Positieve waarden = input (naar rechts)")
print("   - Negatieve waarden = output (weg van systeem)")
print("   - Totaal moet optellen tot 0 (energiebehoud!)")

### 🧪 Oefening: Maak je eigen Sankey

Probeer een `sankey` te maken voor de `warmtestroom` in het gebouw voor 2023.
- Input: Warmte WP en Warmte Gasketel
- Output: Vloerverwarming en luchtgroep en restwarmte (sww en niet bemeterde radiatoren)

**Tip:** De som van alle flows moet 0 zijn!

In [ ]:
# VUL AAN: maak een sankey op basis van energie_2023


## 6️⃣ Volledig Sankey Diagram: Energiestromen Dunant 1

Nu maken we een volledig Sankey diagram voor het gebouw met:
- **Primaire energie**: Gas + Elektriciteit
- **Conversie**: Gasketel + Warmtepomp
- **Tussenopslag**: BEO-veld
- **Afgifte**: Vloerverwarming, Luchtgroepen, Radiatoren
- **Verliezen**: Rendementverliezen in conversie

### Stap 1: Functie ontwerpen — jaarlijkse energiebalans

Doel: bereken per jaar de belangrijkste energiestromen (input → conversie → opslag → afgifte) om overzichtelijke Sankey-diagrammen te maken.

Wat doet deze functie:
- Neemt een lijst jaren en verschillende dagelijkse tijdreeksen (Pandas DataFrame of Series).
- Aggregreert per jaar de totale energie per categorie en retourneert die totalen als MWh.
- Handelt ontbrekende kolommen veilig af (vult met 0) zodat de functie robuust is voor verschillende datasets.

Inputs:
- years: lijst met jaren (bv. [2022, 2023])
- gasverbruik_kwh: DataFrame/Series met kolom 'gasverbruik' (kWh/dag)
- cal_gasketel_dag, cal_wp_dag, cal_beo_dag, cal_vloer_dag, cal_lucht_dag:
  DataFrames met dagelijkse waarden (kWh/dag). De functie deelt op het einde door 1000 → resultaat in MWh.

Output:
- Dict mapping jaar → dict met keys:
  'Gas', 'Gasketel_warmte', 'WP_warmte',
  'BEO_uit', 'BEO_in',
  'Vloerverwarming_warm', 'Vloerverwarming_koud',
  'Luchtgroepen_warm', 'Luchtgroepen_koud'.

Gebruik:
- Bouw energiebalansen = jaarlijkse_energiebalans(years, gasverbruik_kwh, ...)
- Gebruik energiebalansen[2023] om waarden voor 2023 te halen (in MWh).

In [ ]:
def jaarlijkse_energiebalans(years, gasverbruik_kwh, cal_gasketel_dag, cal_inj_dag, cal_wp_dag, cal_beo_dag, cal_vloer_dag, cal_lucht_dag):
    """Bereken jaarlijkse energiebalansen (in MWh)."""
    resultaten = {}
    for ### in ###:
        mask = lambda df: df.index.year == year
        
        # Bereken basis waarden
        gasketel_warmte = cal_gasketel_dag[mask(cal_gasketel_dag)]['gasketel_warm'].sum() / 1000
        injectie_lt = ###[mask(####)]['###'].sum() / 1000
        
        resultaten[year] = {
            # INPUT (primaire energie)
            'Gas': ###.sum() / 1000,

            # CONVERSIE (warmte-opwekking)
            'Gasketel_warmte': ###,
            'Injectie_lt': ###, 
            'WP_warmte': ###,

            # BEO (energieopslag)
            'BEO_uit': ###,
            'BEO_in': ###,

            # AFGIFTE (waar gaat warmte naartoe)
            # TODO
            
            # REST: Radiatoren = gasketel warmte die NIET naar LT-circuit gaat (herbereken voor de zekerheid)
            'Radiatoren_Lucht': gasketel_warmte - injectie_lt  
        }
    return resultaten

# Gebruik (let op: cal_inj_dag toegevoegd!)
years = [2022, 2023, 2024, 2025]
energiebalansen = jaarlijkse_energiebalans(
    years, gasverbruik_kwh, cal_gasketel_dag, cal_inj_dag, cal_wp_dag, cal_beo_dag, cal_vloer_dag, cal_lucht_dag
)

# Test
print(energiebalansen[2023].keys())

### Stap 2: Test de functie voor het jaar 2023

Nu gaan we onze nieuwe functie testen door het Sankey diagram van 2023 opnieuw te maken, maar nu met data uit de functie.

In [ ]:
# Haal de energie-data voor 2023 op uit onze functie
# VUL AAN

### Stap 3: Sankey voor alle jaren tegelijk

Nu we weten dat onze functie werkt, kunnen we eenvoudig `Sankey` diagrammen maken voor **alle jaren** in één keer!

**Voordeel**: We zien meteen trends over de jaren heen (Bijv. verminderd gasverbruik, toegenomen WP-gebruik).

In [ ]:
# Maak een grid van 2x2 subplots (4 jaren)
# TODO -> maak het juiste aantal subplots
axes = axes.flatten()  # Omzetten naar 1D array voor makkelijk indexeren

for idx, year in enumerate(years):
    # Haal data op voor dit jaar
    energie = ###
    gasketel = ###
    wp = ###
    vloerverwarming = ###
    luchtgroep = ###
    verschil = ###
    
    # Maak Sankey in deze subplot
    # TODO 

# Verwijder lege subplots (als er minder dan 4 jaren zijn)
for j in range(len(years), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Vergelijking Warmtestromen 2022-2025', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print(f"✓ Sankey diagrammen voor {len(years)} jaren gemaakt")

### 💡 Reflectievragen

- Zie je een trend in de opwekking over de jaren?
- Zie je een trend in de afgifte over de jaren?
- Welke informatie zou je nog kunnen toevoegen om de trends beter te analyseren?

### Stap 4: Volledige systeem

Tot nu toe hebben we alleen de **warmte-output** (gasketel + WP) naar de afgiftesystemen gevisualiseerd.

Maar nu gaan we het **volledige systeem** in kaart brengen

```PlainText
GAS → GASKETEL → WARMTE 
                    ↓
              DISTRIBUTIE → VLOER + LUCHT + REST
                    ↑
BEO ← WARMTEPOMP ← ELEKTRICITEIT
```

Dit is complexer omdat we **meerdere Sankey-lagen** na elkaar moeten koppelen.

#### 💭 Begrijp je nu de volledige keten?

Gas → Gasketel → Warmte → Distributie → Gebouw  
Elektriciteit → WP (+ BEO) → Warmte → Distributie → Gebouw

#### Vereenvoudig volledig Sankey (zonder BEO detail)

We beginnen met een **versimpelde versie** zonder BEO-details, om de basis te leggen:

In [ ]:
# Data voor 2023
e = energie_2023.copy()

# Destileer de verschillende stromen (gasketel, warmtepomp, injectie lt, ...)


# BELANGRIJK: Bereken wat er overblijft voor niet-bemeterde systemen
# (verschil tussen injectie + WP warmte en gemeten afgifte)
rest = 

# Maak diagram




# LAAG 1: Gasketel → splits naar LT-circuit en radiatoren
sankey.add(
    flows=[gasketel_warmte, -injectie, -radiatoren],
    labels=[f'Gasketel\n{gasketel_warmte:.0f} MWh', '', f'Radiatoren\n{radiatoren:.0f} MWh'],
    orientations=[0, 0, 1],  # Gasketel rechts, injectie rechts, radiatoren omhoog
    facecolor='orange',
    alpha=0.6,
    patchlabel='Gasketel'
)

# LAAG 2: LT-distributie → vloer + lucht + rest
sankey.add(
    # TODO
)

diagrams = sankey.finish()

plt.title('Energie-Sankey 2023\nGasketel → Radiatoren + LT-circuit (+ WP) → Gebouw', 
          fontsize=15, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 7️⃣ Interactief Sankey Diagram met Plotly

Matplotlib Sankey diagrammen zijn statisch. Voor **interactieve rapportage** gebruiken we **Plotly**, waarmee je:
- Over stromen kunt hoveren voor details
- Kunt inzoomen en pannen
- Makkelijk kunt exporteren naar HTML

In [ ]:
# Indien foutmeldingen bij volgende codeblok, installeer juiste versie nbformat en herstart
%pip install nbformat>=4.2.0

In [ ]:
# Plotly Sankey: Start vanuit warmte-opwekking (niet primair gas)
e = energie_2023.copy()

# Bereken stromen
gasketel_warmte = e['Gasketel_warmte']
injectie = e['Injectie_lt']
radiatoren = e['Radiatoren_Lucht']
wp_warmte = e['WP_warmte']
vloer = e['Vloerverwarming_warm']
lucht = e['Luchtgroepen_warm']
rest = (injectie + wp_warmte) - (vloer + lucht)

# Definieer nodes (0-indexed) - NU ZONDER GAS INPUT
nodes = [
    "Gasketel",            # 0 - START HIER
    "Warmtepomp",          # 1 - START HIER
    "Injectie LT",         # 2
    "Radiatoren & NVWB",   # 3
    "LT-Distributie",      # 4
    "Vloerverwarming",     # 5
    "Luchtgroepen",        # 6
    "Rest (SWW, etc.)"     # 7
]

# Definieer links (bron → bestemming, waarde)
links = {
    'source': [
        0,  # Gasketel → Injectie LT
        0,  # Gasketel → Radiatoren
        2,  # Injectie LT → LT-Distributie
        1,  # WP → LT-Distributie
        4,  # LT-Distributie → Vloer
        4,  # LT-Distributie → Lucht
        4   # LT-Distributie → Rest
    ],
    'target': [
        2,  # → Injectie LT
        3,  # → Radiatoren
        4,  # → LT-Distributie
        4,  # → LT-Distributie
        5,  # → Vloer
        6,  # → Lucht
        7   # → Rest
    ],
    'value': [
        injectie,
        radiatoren,
        injectie,
        wp_warmte,
        vloer,
        lucht,
        rest
    ],
    'label': [
        f"{injectie:.0f} MWh",
        f"{radiatoren:.0f} MWh",
        f"{injectie:.0f} MWh",
        f"{wp_warmte:.0f} MWh",
        f"{vloer:.0f} MWh",
        f"{lucht:.0f} MWh",
        f"{rest:.0f} MWh"
    ]
}

# Maak Plotly Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=20,
        thickness=25,
        line=dict(color="black", width=0.5),
        label=[
            f"Gasketel<br>{gasketel_warmte:.0f} MWh",
            f"Warmtepomp<br>{wp_warmte:.0f} MWh",
            "Injectie LT",
            "Radiatoren & NVWB",
            "LT-Distributie",
            "Vloerverwarming",
            "Luchtgroepen",
            "Rest (SWW, etc.)"
        ],
        color=[
            "orange",      # Gasketel
            "blue",        # WP
            "darkorange",  # Injectie LT
            "brown",       # Radiatoren
            "lightblue",   # LT-Distributie
            "lightgreen",  # Vloer
            "lightgreen",  # Lucht
            "lightgray"    # Rest
        ]
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        label=links['label'],
        color=[
            "rgba(255,165,0,0.4)",   # Gasketel → Injectie
            "rgba(165,42,42,0.4)",   # Gasketel → Radiatoren
            "rgba(255,140,0,0.4)",   # Injectie → Distributie
            "rgba(0,0,255,0.4)",     # WP → Distributie
            "rgba(144,238,144,0.4)", # Distributie → Vloer
            "rgba(144,238,144,0.4)", # Distributie → Lucht
            "rgba(211,211,211,0.4)"  # Distributie → Rest
        ]
    )
)])

fig.update_layout(
    title="Interactief Energie-Sankey 2023<br><sub>Warmte-opwekking → Distributie → Afgifte</sub>",
    font=dict(size=12),
    height=700,
    plot_bgcolor='white'
)

fig.show()

print("✓ Interactief Plotly Sankey gemaakt (start vanuit warmte-opwekking)")
print(f"\n📊 Stroomoverzicht:")
print(f"  🔥 Gasketel ({gasketel_warmte:.1f} MWh):")
print(f"     → Injectie LT: {injectie:.1f} MWh")
print(f"     → Radiatoren: {radiatoren:.0f} MWh")
print(f"\n  ❄️  Warmtepomp ({wp_warmte:.1f} MWh):")
print(f"     → Direct naar LT-Distributie")
print(f"\n  🏠 LT-circuit ({injectie + wp_warmte:.1f} MWh):")
print(f"     → Vloer: {vloer:.1f} MWh")
print(f"     → Lucht: {lucht:.0f} MWh")
print(f"     → Rest: {rest:.1f} MWh")

> 💡 **TIP!**: Bewaar je diagram als HTML om te delen met anderen.

In [ ]:
fig.write_html("sankey_2023.html")

## 8️⃣ Tijdsafhankelijke Analyse: Maandelijkse Energiestromen

Tot nu toe hebben we jaarlijkse totalen gebruikt. Maar hoe **varieert** de energiestroom doorheen het jaar? Laten we maandelijkse data visualiseren.

### 🎨 Interactieve Maandelijkse Sankey (Bonus)

Voor een nog beter inzicht kunnen we een **dropdown menu** toevoegen om per maand het Sankey diagram te tonen.

In [ ]:
# 🎨 Interactieve Sankey voor alle jaren
figures = []

for year in years:
    # TODO
    
    figures.append(fig)

# Toon alle figuren na elkaar
for idx, fig in enumerate(figures):
    print(f"\n📊 Sankey diagram voor {years[idx]}")
    fig.show()

print(f"\n✓ {len(figures)} interactieve Sankey diagrammen gemaakt voor jaren {years}")

## 🎓 Samenvatting: Wat heb je geleerd?

In deze oefening heb je geleerd om:

1. **Sankey diagrammen maken** met Matplotlib en Plotly
2. **Energiestromen structureren** van input → conversie → opslag → distributie → afgifte
3. **Energiebalansen opstellen** en verliezen identificeren
4. **Rendementen visualiseren** op systeem- en componentniveau
5. **Tijdsafhankelijke patronen** analyseren (maandelijks, jaarlijks)
6. **Interactieve visualisaties** maken voor rapportage
7. **Data-gedreven aanbevelingen** formuleren

### 🔧 Herbruikbare Skills

De technieken die je hier hebt geleerd zijn direct toepasbaar op:
- Andere gebouwen met HVAC-systemen
- Industriële processen (energie, water, materiaalstromen)
- Financiële stromen (cash flow, budgetanalyse)
- Elke situatie waar je **stromen tussen knooppunten** wilt visualiseren

### 📚 Volgende Stappen

Experimenteer verder door:
- Andere jaren te analyseren (2020-2023)
- Vergelijkingen te maken tussen jaren
- Weekpatronen te onderzoeken (werkdag vs weekend)
- Correlaties met buitentemperatuur te visualiseren

---

**💪 Goed gedaan!** Je hebt nu een krachtige toolbox voor energiestromen-analyse.

-----
EINDE

---